本文仅介绍postgis数据库中的路网数据拓扑处理、网络分析函数的创建，及前端展示Geoserver发布的sql view服务的展示。关于运行环境、服务发布、openlayer地图调用部分参见“最短路劲分析”

**创建拓扑函数**

In [ ]:
-- FUNCTION: public.analysis_updatetopologycharacter varying

-- DROP FUNCTION public.analysis_updatetopologycharacter varying;

CREATE OR REPLACE FUNCTION public.analysis_updatetopology(
	tbl character varying,
	differ double precision DEFAULT 0.001)
    RETURNS character varying
    LANGUAGE 'plpgsql'
    COST 100.0
    VOLATILE STRICT 
AS $function$


BEGIN   
    --添加起点id
    -- 检查列 'pgr_source' 是否存在
    IF NOT EXISTS (
        SELECT 1
        FROM information_schema.columns 
        WHERE 
            table_schema = 'public' AND 
            table_name   = tbl AND 
            column_name  = 'pgr_source'
    ) THEN
        -- 如果不存在，添加列
        execute 'ALTER TABLE public.' || tbl || ' ADD COLUMN pgr_source integer';
    END IF;
    
    --添加终点id
    -- 检查列 'pgr_target' 是否存在
    IF NOT EXISTS (
	SELECT 1
	FROM information_schema.columns 
	WHERE 
	    table_schema = 'public' AND 
	    table_name   = tbl AND 
	    column_name  = 'pgr_target'
	) THEN
	-- 如果不存在，添加列
	execute 'ALTER TABLE public.' || tbl || ' ADD COLUMN pgr_target integer';
    END IF;
 
    --添加权重值
    -- 检查列 'pgr_target' 是否存在
    IF NOT EXISTS (
	SELECT 1
	FROM information_schema.columns 
	WHERE 
	    table_schema = 'public' AND 
	    table_name   = tbl AND 
	    column_name  = 'pgr_length'
	) THEN
	-- 如果不存在，添加列
	execute 'ALTER TABLE public.' || tbl || ' ADD COLUMN pgr_length integer';
    END IF;

    --为source字段创建索引
    IF NOT EXISTS (
        SELECT 1 FROM pg_indexes 
        WHERE schemaname = 'public' AND indexname = 'pgr_source_idx'
    ) THEN
        execute 'CREATE INDEX pgr_source_idx ON public.' || tbl || '("pgr_source")';
    END IF;
    
    --为target字段创建索引
    IF NOT EXISTS (
        SELECT 1 FROM pg_indexes 
        WHERE schemaname = 'public' AND indexname = 'pgr_target_idx'
    ) THEN
        execute 'CREATE INDEX pgr_target_idx ON public.' || tbl || '("pgr_target")';
    END IF;
    
    --为目标表创建拓扑布局，即为source和target字段赋值
    execute 'select public.pgr_createTopology(''public.' || tbl || ''',' || differ || ', ''geom'', ''gid'', ''pgr_source'', ''pgr_target'')';
    --为length赋值，geom为几何类型的字段，可能为shape、the_geom
    execute 'update public.' || tbl || ' set pgr_length = public.st_length(geom)';
    --执行完成
    RETURN 'OK';
    
EXCEPTION
    WHEN OTHERS THEN
        -- 错误处理
        RETURN 'Error: ' || SQLERRM;
END;
$function$;

ALTER FUNCTION public.analysis_updatetopology(character varying,double precision)
    OWNER TO postgres;

In [ ]:
--调用

SELECT analysis_connect('pipe', 0.001);

**创建网络分析函数**

In [ ]:
-- Function: public.analysis_connect(character varying, double precision, double precision, double precision, double precision, double precision)

-- DROP FUNCTION public.analysis_connect(character varying, double precision, double precision, double precision, double precision, double precision);

CREATE OR REPLACE FUNCTION public.analysis_connect(
    tbl character varying,
    startx double precision,
    starty double precision,
    endx double precision,
    endy double precision,
    differ double precision)
  RETURNS SETOF pipe AS
$BODY$

DECLARE

    v_startTarget integer;--距离起点最近点   
    v_endSource integer;--距离终点最近点 
    v_SRID integer;-- 获取当前坐标系SRID
    v_startPoint VARCHAR (255);--选择的起点
    v_endPoint VARCHAR (255);--选择的终点

BEGIN

    -- 获取当前坐标系SRID
    execute 'select ST_SRID(the_geom) from '||tbl||'_vertices_pgr where id=1' into v_SRID;
    -- 定义起点坐标
    v_startPoint = 'public.ST_GeomFromText(''point('||startx||' '||starty||')'','||v_SRID||')';
    -- 定义终点坐标
    v_endPoint = 'public.ST_GeomFromText(''point('||endx||' '||endy||')'','||v_SRID||')';

    --查询离起点最近的点,differ为容差值
    execute 'select id  from '||tbl||'_vertices_pgr where   
            public.ST_DWithin(the_geom,'||v_startPoint||','||differ||')   
            order by public.ST_Distance(the_geom,'||v_startPoint||') limit 1'  
            into v_startTarget;   
    --查询离终点最近的点  
    execute 'select id  from '||tbl||'_vertices_pgr where   
            public.ST_DWithin(the_geom,'||v_endPoint||','||differ||')   
            order by public.ST_Distance(the_geom,'||v_endPoint||') limit 1'  
            into v_endSource;

    --如果没找到最近的点，就返回null   
    if (v_startTarget is null) or (v_endSource is null) then   
        if (v_startTarget is null) THEN
          raise notice '没有找到起点';
        end if;
        if (v_endSource is null) THEN
          raise notice '没有找到终点';
        end if;
        v_startTarget=0; 
        v_endSource=0;   
    end if ;

    RETURN  QUERY 
        execute 'SELECT  b.* 
                FROM public.pgr_kdijkstraPath(
                ''SELECT gid as id, pgr_source as source, pgr_target as target, pgr_length as cost 
                FROM '||tbl||' where geom is not null'','    
                ||v_startTarget||', '||'array['||v_endSource||'] , false, false    
                ) a, '||tbl||' b    
                WHERE a.id3=b.gid
                ORDER by id1';

END 


$BODY$
  LANGUAGE plpgsql VOLATILE
  COST 100
  ROWS 1000;
ALTER FUNCTION public.analysis_connect(character varying, double precision, double precision, double precision, double precision, double precision)
  OWNER TO postgres;



In [ ]:
--调用

SELECT analysis_updatetopology('pipe',13722126.6941279,5036398.09328714,13721542.815,5023826.240,300);

**SQL View**

GeoServer 可以发布 SQL View 作为 WMS 或 WFS 服务。SQL View 是一种虚拟图层，是通过执行一个 SQL 查询来动态生成数据。

openlayer调用WMS服务


In [ ]:
//openlayer v4.6.5
//返回值是多条记录的时候也可以用
var startCoord = [13722126.6941279,5036398.09328714];
var destCoord = [13693143.583109,4995007.75154479];
var params = {
    LAYERS: 'NA:func_analysis_connect',
    FORMAT: 'image/png',
};
var viewparams = [
    'x1:' + startCoord[0], 'y1:' + startCoord[1],
    'x2:' + destCoord[0], 'y2:' + destCoord[1]
];
console.log(viewparams);
params.viewparams = viewparams.join(';');
result = new ol.layer.Image({
    source: new ol.source.ImageWMS(
        {
            url: 'http://localhost:8080/geoserver/NA/wms',
            params: params
        })
});

ajax访问 WFS服务

In [ ]:
//url
//http://localhost:8080/geoserver/NA/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=NA%3Afunc_analysis_connect&maxFeatures=50&outputFormat=application%2Fjson&viewparams=x1:13722126.6941279;y1:5036398.09328714;x2:13693143.583109;y2:4995007.75154479

        var vectorSource = new ol.source.Vector();
        var mystyle = new ol.style.Style({
            stroke: new ol.style.Stroke({
                color: "#ffff00",
                width:2
            })
        });

        //定义矢量图层
        var vectorLayer = new ol.layer.Vector({
        source: vectorSource,
        name: 'vector',
        style:mystyle
        });
        map.addLayer(vectorLayer);

        // 定义额外的参数
      var startCoord = [13722126.6941279,5036398.09328714];
      var destCoord = [13693143.583109,4995007.75154479];
      var viewparams = [
          'x1:' + startCoord[0], 'y1:' + startCoord[1],
          'x2:' + destCoord[0], 'y2:' + destCoord[1]
      ];
      var wfsUrl = 'http://localhost:8080/geoserver/wfs'; // 替换为 GeoServer WFS 的实际 URL
      $.ajax({
          url: wfsUrl,
          type: 'GET',
          data: {
              'SERVICE': 'WFS',
              'VERSION': '1.0.0', // WFS 版本，根据你的 GeoServer 版本进行调整
              'REQUEST': 'GetFeature',
              'TYPENAME': 'NA:func_analysis_connect', // SQL View 的名称，包含工作空间前缀
              'OUTPUTFORMAT': 'application/json', // 输出格式，这里使用 JSON
              'viewparams': viewparams.join(';')
          }, // 使用 $.extend 合并基本参数和额外参数
          dataType: 'json',
          success: function(data) {
                // 请求成功，处理返回的 GeoJSON 数据
                console.log('Features retrieved:', data);
                var json = data;
                features = new ol.format.GeoJSON().readFeatures(json);
                vectorSource.addFeatures(features);
          },
          error: function(jqXHR, textStatus, errorThrown) {
              // 请求失败，处理错误情况
              console.error('WFS request failed: ' + textStatus + ', ' + errorThrown);
          }
      });


**参考**

[GeoServer系列之SqlView](https://www.cnblogs.com/hanhuibing/articles/5642703.html)

[PostGIS管网连通性分析](https://blog.csdn.net/gisarmory/article/details/116194268)